# Job search

Compute the continuation value as a function of $\beta$.

In [1]:
import numpy as np
from scipy.stats import betabinom
from collections import namedtuple
import matplotlib.pyplot as plt

In [2]:
def successive_approx(T,                     # Operator (callable)
                      x_0,                   # Initial condition
                      tolerance=1e-6,        # Error tolerance
                      max_iter=10_000,       # Max iteration bound
                      print_step=25,         # Print at multiples
                      verbose=False):        
    x = x_0
    error = np.inf
    k = 1
    while error > tolerance and k <= max_iter:
        x_new = T(x)
        error = np.max(np.abs(x_new - x))
        if verbose and k % print_step == 0:
            print(f"Completed iteration {k} with error {error}.")
        x = x_new
        k += 1
    if k < max_iter:
        print(f"Terminated successfully in {k} iterations.")
    else:
        print(f"Warning: Iteration hit max_iter bound {max_iter}.")
    return x

In [3]:
Model = namedtuple("Model", ("w_vals", "φ", "β", "c")) 

In [4]:
def create_job_search_model(
        n=50,        # wage grid size
        w_min=10.0,  # lowest wage
        w_max=60.0,  # highest wage
        a=200,       # wage distribution parameter
        b=100,       # wage distribution parameter
        β=0.96,      # discount factor
        c=10.0       # unemployment compensation
    ):
    w_vals = np.linspace(w_min, w_max, n+1)
    φ = np.array([betabinom(n, a, b).pmf(k) for k in range(n+1)])
    return Model(w_vals=w_vals, φ=φ, β=β, c=c)

The function $g(h) = c + \beta \sum_{w'} \max\{w'/(1-\beta), h\} \phi(w')$.  The fixed point is the continuation value.

In [5]:
def g(h, model):
    w_vals, φ, β, c = model.w_vals, model.φ, model.β, model.c
    return c + β * np.maximum(w_vals / (1 - β), h) @ φ

Task: Plot the fixed point $h^*$ of $g$ as a function of $\beta$.  Use the values

In [6]:
betas = np.linspace(0.9, 0.99, 20) 